# Import materials from uploaded files

Upload files with materials structure data and convert them to ESSE format.

<h2 style="color:green">Usage</h2>

1. Upload files to `user_uploads` folder
1. Click "Run" > "Run All" to run all cells
1. In case of format detection error, please check the file format extension and try again

## Methodology

The following happens in the script below:

1. Install the required packages
2. Import the material from files and read the structure data into ASE Atoms object
3. Convert the ASE Atoms objects to ESSE format and visualize the structures
4. Pass the materials to the outside runtime

## 1. Set Parameters

In [2]:
FOLDER_PATH = "user_uploads"

## 2. Install Packages

In [ ]:
from jupyterlite.utils import install_packages

await install_packages("import_material_from_files.ipynb")

## 3. Data Processing

### 3.1. Read materials from files using ASE

In [ ]:
import os
from ase.io import read

materials = []
unreadable_files = []

for file_name in os.listdir(FOLDER_PATH):
    file_path = os.path.join(FOLDER_PATH, file_name)
    try:
        atoms = read(file_path)
        materials.append(atoms)
        
    except Exception as e:
        print(e)
        unreadable_files.append(file_name)
        continue

### 3.2. Preview the data

In [ ]:
print(f"Successfully read {len(materials)} files")
print(f"Unreadable files: {unreadable_files}")

### 3.3. Filter data

In [ ]:
# Filter materials if needed

### 3.4. Convert to ESSE format

In [ ]:
from src.utils import ase_to_poscar
from express import ExPrESS

def convert_ase_entry_to_esse(ase_entry):
    poscar = ase_to_poscar(ase_entry)
    kwargs = {
    "structure_string": poscar,
    "structure_format": "poscar"
    }

    handler = ExPrESS("structure", **kwargs)
    esse = handler.property("material", **kwargs)
    return esse

esse_entries = list(map(convert_ase_entry_to_esse, materials))

### 3.5. Preview the data

In [ ]:
from src.utils import poscar_to_ase
from ase.visualize import view
from ase.io import write
from ase.build import make_supercell
from IPython.display import Image

def visualize_material(material, index: int, number_of_repetitions: int = 3):
    """
    Visualize the material using ASE's visualization tool
    Repeat the unit cell to make it easier to see.

    Args:
        material: The material to visualize (Ase.Atoms object)
        index: The index of the material
        number_of_repetitions: The number of unit cell repetitions to visualize
    """
    # Set the number of unit cell repetition for the structure to make it easier to see
    n = number_of_repetitions
    material_repeat = make_supercell(material, [[n,0,0],[0,n,0],[0,0,n]])
    filename = f"material-{index}.png"
    write(filename, material_repeat)
    img = Image(filename=filename)
    print(filename, "-", material.symbols)
    display(img)

for idx, material in enumerate(materials):
    visualize_material(material, idx)

## 4. Pass data to the outside runtime

In [ ]:
from jupyterlite.utils import set_data

output_materials = esse_entries
set_data("materials", output_materials)